# Real images augmentation (moving and flipping)

State:
- We have a annotated testset (around 50 images right now)

Goal:
- We want to move objects in the real images and flip them, while also moving the bouding boxes - in order to get a bigger test set

In [5]:
PATH_IMGS = "/home/jetracer/Documents/3d_mai/data/real_images/4_real_images/preprocessed-resized/annotated/augmented/testing/images"

# Set the region of where the objects can be moved to
MIN_X = 250
MAX_X = 550
MIN_Y = 150
MAX_Y = 300

# Flipping probability
FLIP_PROB = 0.5

# Augmentation number (1 means one augmentation per image, resulting in 2x the original dataset)
AUGMENTATION_NUM = 1

### Moving and flipping randomly

- Move objects and flip images randomly.
- Don't affect bounding boxes. The newly created images have to be labeled manually after. (This decision was done, since there would be too many problems otherwise.)

In [6]:
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
import random

def get_files_in_subdirectories(folder_path, file_extension='', file_contains=''):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if file_extension == '' and file_contains == '':
                files.append(os.path.join(root, filename))
            elif file_extension != '' and file_contains == '':
                if filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
            elif file_extension == '' and file_contains != '':
                if file_contains in filename:
                    files.append(os.path.join(root, filename))
            else:
                if file_contains in filename and filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
    return files

images_move = get_files_in_subdirectories(PATH_IMGS,'.jpg')
unique_classes = []

def move_and_flip(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the convex hull of the contours
    convex_hull = cv2.convexHull(np.vstack(contours))

    # Create a copy of the image for drawing the bounding box
    result_image = image.copy()

    # Calculate the bounding box of the convex hull
    x, y, w, h = cv2.boundingRect(convex_hull)

    image_width = image.shape[1]
    image_height = image.shape[0] 

    aratio = w/h
    if aratio >= 1:
        px = 1
        py = int(1*aratio)
    else:
        px = int(1.0/aratio)
        py = 1
    x1 = x-px
    y1 = y-py
    x2 = x + w + px
    y2 = y + h + py
    wn = x2-x1
    hn = y2-y1
    
    # Set the region, where the object can be pasted to.
    new_x = np.random.randint(MIN_X, MAX_X + 1)
    new_y = np.random.randint(MIN_Y, MAX_Y + 1)

    # Extract the object
    object_region = image[y1:y2, x1:x2].copy()

    # Paste the object into the new random location
    image[y1:y2, x1:x2] = [0, 0, 0]
    image[new_y:new_y+hn, new_x:new_x+wn] = object_region


    # FLIPPING
    # Potentially flip vertically
    if random.random() < FLIP_PROB:
        image = cv2.flip(image, 0)  # Flip vertically

    # Potentially flip horizontally
    if random.random() < FLIP_PROB:
        image = cv2.flip(image, 1) # Flip horizontally

    return image

    
print("Number of images found:", len(images_move))
print("Augmenting images...")
for index, jpg in enumerate(images_move):
    filename = jpg.split('/')[-1].split('.')[0]
    new_filename = filename + "_moved_flipped"
    new_jpg_path = jpg.replace(filename, new_filename)

    # Load the image
    image = cv2.imread(jpg)

    for i in range(AUGMENTATION_NUM):
        # Potentially move and flip
        image = move_and_flip(image)

        # Save the image
        cv2.imwrite(new_jpg_path.replace("_moved_flipped", "_moved_flipped_" + str(i)), image)

print("Number of resulting images:", len(images_move) + len(images_move) * AUGMENTATION_NUM)


Number of images found: 45
Augmenting images...
Number of resulting images: 90
